In [ ]:
print("hello")


## project started 
Demo time

In [ ]:
import os
import requests
from dotenv import load_dotenv
import json
from IPython.display import Markdown, display
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


In [ ]:
model="gpt-4o-mini"
openai = OpenAI()
TAVILY_API_KEY=os.getenv('TAVILY_API_KEY')

In [ ]:
system_prompt = """You are FarmSakha AI – a multilingual, AI-powered Smart Crop Advisory Assistant designed to help small and marginal farmers in India. 
Your role is to provide personalized, real-time agricultural guidance in simple and easy-to-understand language.

Core Responsibilities:
1. Crop Advisory:
   - Suggest suitable crops based on soil type, weather conditions, season, and historical crop data.
   - Provide scientific fertilizer and irrigation recommendations with correct NPK ratios.

2. Pest and Disease Management:
   - Help farmers identify pests/diseases through image descriptions or text inputs.
   - Suggest eco-friendly and cost-effective solutions, including biological and chemical control methods.

3. Weather & Risk Alerts:
   - Provide localized, real-time weather updates and 7-day forecasts using coordinates.
   - Warn about risks like drought, floods, or pest outbreaks and suggest preventive measures.
   - Help farmers plan irrigation, spraying, and harvesting based on weather conditions.

4. Soil Health:
   - Give soil test interpretation and fertilizer/lime/organic manure guidance.
   - Promote sustainable practices that reduce chemical overuse.

5. Market & Economics:
   - Share local mandi/market prices of crops using real-time search.
   - Provide insights on which crops may yield better income for the season.

6. Accessibility:
   - Support answers in multiple Indian languages.
   - Provide step-by-step, voice-friendly, and simple responses for low-literate farmers.

Tools Available:
- Real-time Google search for market prices, news, and current information
- Weather forecasting using GPS coordinates (latitude/longitude)

When farmers ask about weather:
- If they provide coordinates, use the weather tool directly
- If they provide city/village names, ask for coordinates or use your knowledge to provide general guidance
- Always relate weather information to farming activities (when to irrigate, spray, harvest, etc.)

Important Guidelines:
- Always prioritize sustainable, cost-effective, and farmer-friendly practices.
- Avoid overly technical jargon – explain in simple words with examples.
- When uncertain, advise consulting local agriculture officers or Krishi Vigyan Kendras.
- Ensure trustworthiness, cultural sensitivity, and clarity in communication.

Your mission: Empower small and marginal farmers with actionable insights, 
improve productivity, reduce costs, and promote environmentally friendly farming practices.


"""


In [ ]:
# def chat(message, history):
#     messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
#     response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

#     if response.choices[0].finish_reason=="tool_calls":
#         message = response.choices[0].message
#         response, city = handle_tool_call(message)
#         messages.append(message)
#         messages.append(response)
#         response = openai.chat.completions.create(model=MODEL, messages=messages)
    
#     return response.choices[0].message.content

In [ ]:
def get_Google_Search(query: str):
        url = "https://api.tavily.com/search"
        headers = {"Content-Type": "application/json"}
        payload = {
            "api_key":TAVILY_API_KEY ,
            "query": query,
            "search_depth": "basic",
            "include_answer": True
        }
        response = requests.post(url, headers=headers, json=payload)
        data = response.json()

        answer = data.get("answer", "")
        results = "\n".join(
            [f"- {r['title']} ({r['url']})" for r in data.get("results", [])]
        )
        return f"{answer}\n\nSources:\n{results}"

get_Google_Search("what is the price of wheat today in india")

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

Google_Search = {
    "name": "get_Google_Search",
    "description": "Search Google for real-time information (news, crops, weather, market prices, etc.)",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query string",
            },
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

weather forcast


In [ ]:
def get_Weather_forecast(latitude: float, longitude: float):
    """Fetch weather forecast for a given latitude and longitude"""
    import requests
    import pandas as pd
    from datetime import datetime
    
    try:
        weather_url = "https://api.open-meteo.com/v1/forecast"
        weather_params = {
            "latitude": latitude,
            "longitude": longitude,
            "daily": [
                "temperature_2m_max", "temperature_2m_min",
                "precipitation_sum", "rain_sum", "snowfall_sum",
                "windspeed_10m_max", "windgusts_10m_max",
                "winddirection_10m_dominant",
                "sunrise", "sunset",
                "et0_fao_evapotranspiration"
            ],
            "hourly": [
                "temperature_2m", "relative_humidity_2m", "dewpoint_2m",
                "apparent_temperature", "precipitation", "rain",
                "cloudcover", "windspeed_10m", "winddirection_10m",
                "surface_pressure", "visibility"
            ],
            "forecast_days": 7,  # 7-day forecast
            "timezone": "auto"
        }
        
        response = requests.get(weather_url, params=weather_params)
        weather_data = response.json()
        
        if "daily" not in weather_data:
            return "Weather data not available for the given coordinates."
        
        # Format daily forecast
        forecast_text = f"📅 **7-Day Weather Forecast** for coordinates ({latitude}, {longitude}):\n\n"
        
        daily = weather_data["daily"]
        for i in range(min(7, len(daily["time"]))):
            date = daily["time"][i]
            tmax = daily["temperature_2m_max"][i]
            tmin = daily["temperature_2m_min"][i]
            rain = daily["rain_sum"][i]
            wind = daily["windspeed_10m_max"][i]
            
            forecast_text += f"**{date}:**\n"
            forecast_text += f"  🌡️ Temperature: {tmin}°C - {tmax}°C\n"
            forecast_text += f"  🌧️ Rainfall: {rain} mm\n"
            forecast_text += f"  💨 Max Wind: {wind} km/h\n\n"
        
        # Add today's hourly forecast (next 12 hours)
        forecast_text += "\n🕐 **Today's Hourly Forecast (Next 12 Hours):**\n"
        hourly = weather_data["hourly"]
        for i in range(min(12, len(hourly["time"]))):
            time = hourly["time"][i]
            temp = hourly["temperature_2m"][i]
            humidity = hourly["relative_humidity_2m"][i]
            rain = hourly["precipitation"][i]
            
            # Format time nicely
            dt = datetime.fromisoformat(time.replace('Z', '+00:00'))
            time_str = dt.strftime('%H:%M')
            
            forecast_text += f"**{time_str}:** {temp}°C, {humidity}% humidity, {rain}mm rain\n"
        
        return forecast_text
        
    except Exception as e:
        return f"Error fetching weather data: {str(e)}"

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

Weather_forcast = {
    "name": "get_Weather_forcast",
    "description": "Fetch weather forecast for a given latitude and longitude (farmer location)(by hourly for today and daily for next 14 days)",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number", "description": "Latitude of the location"},
            "longitude": {"type": "number", "description": "Longitude of the location"}
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": Google_Search},
        {"type":"function","function":Weather_forcast}]

In [ ]:
def handle_tool_call(message):
    """Handle tool calls from OpenAI"""
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    
    if function_name == "get_Google_Search":
        query = arguments.get('query')
        search_result = get_Google_Search(query)
        result_content = search_result
        
    elif function_name == "get_Weather_forecast":
        latitude = arguments.get('latitude')
        longitude = arguments.get('longitude')
        weather_result = get_Weather_forecast(latitude, longitude)
        result_content = weather_result
        
    else:
        result_content = f"Unknown function: {function_name}"
    
    response = {
        "role": "tool",
        "content": result_content,
        "tool_call_id": tool_call.id
    }
    return response

In [ ]:
def chat(message, history):
    """Main chat function for Gradio interface"""
    # Convert Gradio history format to OpenAI format
    messages = [{"role": "system", "content": system_prompt}]
    
    for human_msg, ai_msg in history:
        messages.append({"role": "user", "content": human_msg})
        if ai_msg:  # Only add if AI response exists
            messages.append({"role": "assistant", "content": ai_msg})
    
    messages.append({"role": "user", "content": message})
    
    # Make initial API call
    response = openai.chat.completions.create(
        model=model, 
        messages=messages, 
        tools=tools
    )

    # Handle tool calls if present
    if response.choices[0].finish_reason == "tool_calls":
        assistant_message = response.choices[0].message
        tool_response = handle_tool_call(assistant_message)
        
        # Add tool call and response to messages
        messages.append({
            "role": "assistant",
            "content": assistant_message.content,
            "tool_calls": assistant_message.tool_calls
        })
        messages.append(tool_response)
        
        # Make final API call
        final_response = openai.chat.completions.create(
            model=model, 
            messages=messages
        )
        return final_response.choices[0].message.content
    
    return response.choices[0].message.content